In [1]:
from urllib.request import urlopen
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from ekonlpy.tag import Mecab

# KOSPI200 Data

## Refine Excel_data from Data-Guide

In [2]:
# Data-Guide에서 다운받은 "일별 KOSPI200" 편입여부 data
df = pd.read_excel(r"C:\Users\PC\OneDrive\Data Science\Weak-supervision\생존편향 제거\완료된 데이터\KOSPI200_편입여부.xlsx")
df

,Unnamed: 0,Last Refresh: 2020-02-24 15:26:51,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 5181,Unnamed: 5182,Unnamed: 5183,Unnamed: 5184,Unnamed: 5185,Unnamed: 5186,Unnamed: 5187,Unnamed: 5188,Unnamed: 5189,Unnamed: 5190
0,Calendar Basis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Portfolio,[KSE+KOSDAQ[상장폐지 포함](I.00P)],Default,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Item,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Frequency,일간,KRW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Non-Trading Day,NaN,Asc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3660,2019-12-27 00:00:00,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,N,N,N,N,N,N,200.0
3661,2019-12-28 00:00:00,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,N,N,N,N,N,N,200.0
3662,2019-12-29 00:00:00,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,N,N,N,N,N,N,200.0
3663,2019-12-30 00:00:00,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,N,N,N,N,N,N,200.0


In [6]:
# drop useless part
df = df.iloc[7:,:]
df.drop([9, 10, 11, 12], inplace=True)

C:\Users\PC\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
df

,Unnamed: 0,Last Refresh: 2020-02-24 15:26:51,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 5181,Unnamed: 5182,Unnamed: 5183,Unnamed: 5184,Unnamed: 5185,Unnamed: 5186,Unnamed: 5187,Unnamed: 5188,Unnamed: 5189,Unnamed: 5190
7,Symbol,A000010,A000011,A000012,A000020,A000021,A000022,A000023,A000030,A000031,...,A950030,A950070,A950100,A950110,A950130,A950140,A950160,A950170,A950180,NaN
8,Symbol Name,조흥은행,조흥은행(1신),조흥은행(2신),동화약품,동화약품공업(1신),동화약품(전환1신),동화약품(전환2신),우리은행,한빛은행(1신),...,네프로아이티,중국고섬,SBI모기지,SBI핀테크솔루션즈,엑세스바이오,잉글우드랩,코오롱티슈진,JTC,SNK,NaN
13,2010-01-01 00:00:00,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,...,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.0
14,2010-01-02 00:00:00,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,...,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.0
15,2010-01-03 00:00:00,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,...,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3660,2019-12-27 00:00:00,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,N,N,N,N,N,N,200.0
3661,2019-12-28 00:00:00,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,N,N,N,N,N,N,200.0
3662,2019-12-29 00:00:00,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,N,N,N,N,N,N,200.0
3663,2019-12-30 00:00:00,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,N,N,N,N,N,N,200.0


In [8]:
def length_adjust(str_) :                   # if month, day is single number, convert to two-numbers (ex. 6 → 06, 7 → 07)
    return (2-len(str_)) * '0' + str_ 

def date_convertor(date) :                  # ex. '2010-01-01' → '20100101'
    year = length_adjust(str(date.year))
    month = length_adjust(str(date.month))
    day = length_adjust(str(date.day))
    yyyymmdd = int(year+month+day)
    return yyyymmdd                       # return 'int' type

In [9]:
lv2_index = pd.date_range(start='20100101', end='20191231')
lv2_index = list(map(date_convertor,lv2_index))   # date_convertor 함수를 적용하여 모든 datetime 변환
lv2_index = list(map(str, lv2_index))    # date_convertor로 return된 값은 int이므로 df에 담기 위해 str로 바꿔줌
df.loc[13:,'Unnamed: 0'] = lv2_index

C:\Users\PC\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [10]:
df

,Unnamed: 0,Last Refresh: 2020-02-24 15:26:51,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 5181,Unnamed: 5182,Unnamed: 5183,Unnamed: 5184,Unnamed: 5185,Unnamed: 5186,Unnamed: 5187,Unnamed: 5188,Unnamed: 5189,Unnamed: 5190
7,Symbol,A000010,A000011,A000012,A000020,A000021,A000022,A000023,A000030,A000031,...,A950030,A950070,A950100,A950110,A950130,A950140,A950160,A950170,A950180,NaN
8,Symbol Name,조흥은행,조흥은행(1신),조흥은행(2신),동화약품,동화약품공업(1신),동화약품(전환1신),동화약품(전환2신),우리은행,한빛은행(1신),...,네프로아이티,중국고섬,SBI모기지,SBI핀테크솔루션즈,엑세스바이오,잉글우드랩,코오롱티슈진,JTC,SNK,NaN
13,20100101,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,...,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.0
14,20100102,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,...,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.0
15,20100103,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,...,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3660,20191227,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,N,N,N,N,N,N,200.0
3661,20191228,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,N,N,N,N,N,N,200.0
3662,20191229,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,N,N,N,N,N,N,200.0
3663,20191230,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,N,N,N,N,N,N,200.0


파일을 저장하고 불러올 때, 일괄적으로 "encoding='euc-kr'" 인자를 넣어줌으로써 엑셀 내에서도 자료를 볼 수 있음.

In [23]:
# save the refined data
df.to_csv(r'C:\Users\PC\OneDrive\Data Science\Weak-supervision\생존편향 제거\완료된 데이터\df.csv', encoding='euc-kr')  

## Make new_df (index = corporate name, columns = date(20100101 ~ 20191231))

In [11]:
new_df = df.set_index("Unnamed: 0").T
new_df.loc['Unnamed: 5190', 'Symbol Name'] = '편입개수'
new_df = new_df.set_index("Symbol Name")
new_df

Unnamed: 0,Symbol,20100101,20100102,20100103,20100104,20100105,20100106,20100107,20100108,20100109,...,20191222,20191223,20191224,20191225,20191226,20191227,20191228,20191229,20191230,20191231
Symbol Name,,,,,,,,,,,,,,,,,,,,,
조흥은행,A000010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
조흥은행(1신),A000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
조흥은행(2신),A000012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
동화약품,A000020,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,N,N,N,N,N,N,N,N,N,N
동화약품공업(1신),A000021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
잉글우드랩,A950140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N
코오롱티슈진,A950160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N
JTC,A950170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N


In [201]:
new_df.to_csv(r'C:\Users\PC\OneDrive\Data Science\Weak-supervision\생존편향 제거\완료된 데이터\new_df_bfconv.csv', encoding='euc-kr')

In [12]:
# convert symbol to eliminate useless 'A' (ex. A000010 -> 000010)
def symbol_convertor(str_):
    return str_[1:]

In [13]:
symbol_list = list(new_df.loc[:'SNK','Symbol'].values)
symbol_conv = list(map(symbol_convertor, symbol_list))
symbol_conv

['000010',
 '000011',
 '000012',
 '000020',
 '000021',
 '000022',
 '000023',
 '000030',
 '000031',
 '000032',
 '000040',
 '000041',
 '000042',
 '000050',
 '000060',
 '000061',
 '000062',
 '000070',
 '000071',
 '000072',
 '000073',
 '000075',
 '000076',
 '000080',
 '000081',
 '000082',
 '000083',
 '000085',
 '000086',
 '000087',
 '000090',
 '000091',
 '000100',
 '000101',
 '000102',
 '000105',
 '000106',
 '000107',
 '000110',
 '000111',
 '000112',
 '000120',
 '000121',
 '000122',
 '000130',
 '000131',
 '000132',
 '000140',
 '000141',
 '000145',
 '000146',
 '000147',
 '000149',
 '000150',
 '000151',
 '000155',
 '000156',
 '000157',
 '000160',
 '000161',
 '000162',
 '000170',
 '000171',
 '000180',
 '000181',
 '000200',
 '000201',
 '000202',
 '000203',
 '000205',
 '000206',
 '000210',
 '000211',
 '000212',
 '000215',
 '000216',
 '000218',
 '000220',
 '000221',
 '000225',
 '000227',
 '000230',
 '000231',
 '000232',
 '000240',
 '000241',
 '000242',
 '000243',
 '000250',
 '000270',
 '000271',

In [14]:
len(symbol_conv)

5189

In [15]:
symbol_name = list(df.loc[8,:])[1:-1]
symbol_name

['조흥은행',
 '조흥은행(1신)',
 '조흥은행(2신)',
 '동화약품',
 '동화약품공업(1신)',
 '동화약품(전환1신)',
 '동화약품(전환2신)',
 '우리은행',
 '한빛은행(1신)',
 '한국상업은행(2신)',
 'KR모터스',
 '효성기계공업(1신)',
 '효성기계공업(2신)',
 '경방',
 '메리츠화재',
 '동양화재(1신)',
 '동양화재(2신)',
 '삼양홀딩스',
 '삼양사(1신)',
 '삼양사(2신)',
 '삼양사(3신)',
 '삼양홀딩스우',
 '삼양사(1우1신)',
 '하이트진로',
 '진로(1신)',
 '진로(2신)',
 '진로(1우2신)',
 '진로우',
 '진로(1우1신)',
 '하이트진로2우B',
 '두산상사',
 '두산상사(1신)',
 '유한양행',
 '유한양행(1신)',
 '유한양행(2신)',
 '유한양행우',
 '유한양행(1우1신)',
 '유한양행2우B',
 '제일은행',
 '제일은행(1신)',
 '제일은행(2신)',
 'CJ대한통운',
 '대한통운(1신)',
 '대한통운(2신)',
 '한일은행',
 '한일은행(1신)',
 '한일은행(2신)',
 '하이트진로홀딩스',
 '하이트맥주(1신)',
 '하이트진로홀딩스우',
 '조선맥주 (우선신)',
 '하이트맥주2우B',
 '하이트맥주3우B',
 '두산',
 '두산(1신)',
 '두산우',
 '오비맥주(1우1신)',
 '두산2우B',
 '대한중석',
 '대한중석(1신)',
 '대한중석 (2신)',
 'LG금속',
 'LG금속(1신)',
 '성창기업지주',
 '성창기업(1신)',
 '대우중공업',
 '대우중공업(1신)',
 '대우중공업(2신)',
 '대우중공업(1우2신)',
 '대우중공우',
 '대우중공업(1우1신)',
 '대림산업',
 '대림산업(1신)',
 '대림산업(전환1신)',
 '대림산업우',
 '대림산업(1우1신)',
 '대림산업(2우1신)',
 '유유제약',
 '유유산업(1신)',
 '유유제약1우',
 '유유제약2우B',
 '일동홀딩스',
 '일동제약(1신)',


In [16]:
len(symbol_name)

5189

In [17]:
# make dictionary for corporate name & corporate code
symbol_dic = {}
for i in range(len(symbol_name)):
    name = symbol_name[i]
    code = symbol_conv[i]
    symbol_dic[name] = code
symbol_dic

{'조흥은행': '000010',
 '조흥은행(1신)': '000011',
 '조흥은행(2신)': '000012',
 '동화약품': '000020',
 '동화약품공업(1신)': '000021',
 '동화약품(전환1신)': '000022',
 '동화약품(전환2신)': '000023',
 '우리은행': '000030',
 '한빛은행(1신)': '000031',
 '한국상업은행(2신)': '000032',
 'KR모터스': '000040',
 '효성기계공업(1신)': '000041',
 '효성기계공업(2신)': '000042',
 '경방': '000050',
 '메리츠화재': '000060',
 '동양화재(1신)': '000061',
 '동양화재(2신)': '000062',
 '삼양홀딩스': '000070',
 '삼양사(1신)': '000071',
 '삼양사(2신)': '000072',
 '삼양사(3신)': '000073',
 '삼양홀딩스우': '000075',
 '삼양사(1우1신)': '000076',
 '하이트진로': '000080',
 '진로(1신)': '000081',
 '진로(2신)': '000082',
 '진로(1우2신)': '000083',
 '진로우': '000085',
 '진로(1우1신)': '000086',
 '하이트진로2우B': '000087',
 '두산상사': '000090',
 '두산상사(1신)': '000091',
 '유한양행': '000100',
 '유한양행(1신)': '000101',
 '유한양행(2신)': '000102',
 '유한양행우': '000105',
 '유한양행(1우1신)': '000106',
 '유한양행2우B': '000107',
 '제일은행': '000110',
 '제일은행(1신)': '000111',
 '제일은행(2신)': '000112',
 'CJ대한통운': '000120',
 '대한통운(1신)': '000121',
 '대한통운(2신)': '000122',
 '한일은행': '000130',
 '한일은행(1신)': '00

In [18]:
len(symbol_dic)

5172

In [19]:
# example of using 'symbol_dic'
symbol_dic['알루코']

'021570'

### symbol_dic의 개수가 17개 더 적은 이유는 symbol_name 중 이름이 동일한 게 17개 있기 때문

In [20]:
len(symbol_name)

5189

In [21]:
len(set(symbol_name))

5172

In [22]:
# symbol을 corporate code에서 'A'를 제거한 symbol_conv로 변환
new_df.loc[:'SNK','Symbol'] = symbol_conv
new_df

Unnamed: 0,Symbol,20100101,20100102,20100103,20100104,20100105,20100106,20100107,20100108,20100109,...,20191222,20191223,20191224,20191225,20191226,20191227,20191228,20191229,20191230,20191231
Symbol Name,,,,,,,,,,,,,,,,,,,,,
조흥은행,000010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
조흥은행(1신),000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
조흥은행(2신),000012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
동화약품,000020,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,N,N,N,N,N,N,N,N,N,N
동화약품공업(1신),000021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
잉글우드랩,950140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N
코오롱티슈진,950160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N
JTC,950170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N


In [23]:
new_df

Unnamed: 0,Symbol,20100101,20100102,20100103,20100104,20100105,20100106,20100107,20100108,20100109,...,20191222,20191223,20191224,20191225,20191226,20191227,20191228,20191229,20191230,20191231
Symbol Name,,,,,,,,,,,,,,,,,,,,,
조흥은행,000010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
조흥은행(1신),000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
조흥은행(2신),000012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
동화약품,000020,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,N,N,N,N,N,N,N,N,N,N
동화약품공업(1신),000021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
잉글우드랩,950140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N
코오롱티슈진,950160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N
JTC,950170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N


In [24]:
type(new_df['Symbol'][0])

str

In [22]:
new_df.to_csv(r'C:\Users\PC\OneDrive\Data Science\Weak-supervision\생존편향 제거\완료된 데이터\new_df.csv', encoding='euc-kr')

In [25]:
new_df.index

Index(['조흥은행', '조흥은행(1신)', '조흥은행(2신)', '동화약품', '동화약품공업(1신)', '동화약품(전환1신)',
       '동화약품(전환2신)', '우리은행', '한빛은행(1신)', '한국상업은행(2신)',
       ...
       '네프로아이티', '중국고섬', 'SBI모기지', 'SBI핀테크솔루션즈', '엑세스바이오', '잉글우드랩', '코오롱티슈진',
       'JTC', 'SNK', '편입개수'],
      dtype='object', name='Symbol Name', length=5190)

데이터프레임을 살펴보니, 동일한 이름의 다른 기업코드를 가진 'SK'가 2개 있었다. (SK(003600), SK(034730))


이후에 진행될 과정에서 MD&A 분석의 편의를 위해 20100101~20191231 전 기간동안 KOSPI200에 편입되어 있는 'perfect_crp'만 이용하여 프로젝트를 진행하고자 한다. 다른 기업코드를 가진 SK 모두 전체 기간동안 코스피200에 편입되어 있었던 게 아니기 때문에 이후에 진행될 과정에서 SK는 데이터에서 제외한다.

In [26]:
new_df.T['SK']

Symbol Name,SK,SK
Unnamed: 0,,
Symbol,003600,034730
20100101,Y,N
20100102,Y,N
20100103,Y,N
20100104,Y,N
...,...,...
20191227,NaN,Y
20191228,NaN,Y
20191229,NaN,Y


## ----------------------데이터가이드에서 받아온 Excel 자료를 깔끔하게 정제한 과정-------------------------

## daily_k200

daily_k200을 만드는 이유는 데이터의 생존편향을 제거해주기 위해 일별로 KOPI200 편입여부가 업데이트된 기업 리스트를 만들기 위해

In [27]:
# row의 길이를 202로 한 이유는 20100101 ~ 20191231의 기간동안 daily로 KOSPI200에 편입된 회사의 개수(199,200,201,202)의 최대가 202개이기 때문
daily_k200 = pd.DataFrame(np.full(shape = (202, len(lv2_index)), fill_value=np.nan), columns = lv2_index)
daily_k200

,20100101,20100102,20100103,20100104,20100105,20100106,20100107,20100108,20100109,20100110,...,20191222,20191223,20191224,20191225,20191226,20191227,20191228,20191229,20191230,20191231
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# 20100101 하루를 기준으로 확인해봤을 때, KOSPI200에 편입된 일별 기업리스트가 index에 나오는 것을 알 수 있음
criteria = new_df.loc[:,'20100101'] == 'Y'
new_df[criteria]

Unnamed: 0,Symbol,20100101,20100102,20100103,20100104,20100105,20100106,20100107,20100108,20100109,...,20191222,20191223,20191224,20191225,20191226,20191227,20191228,20191229,20191230,20191231
Symbol Name,,,,,,,,,,,,,,,,,,,,,
동화약품,000020,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,N,N,N,N,N,N,N,N,N,N
경방,000050,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,N,N,N,N,N,N,N,N,N,N
삼양홀딩스,000070,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
유한양행,000100,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
CJ대한통운,000120,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
후성,093370,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
SK이노베이션,096770,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
한진중공업,097230,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,N,N,N,N,N,N,N,N,N,N


In [29]:
# 20100101 ~ 20191231의 기간동안 일별 KOSPI200 개수는 199, 200, 201, 202개 범위 안에 있음
for d in lv2_index:
    print("Process : {}".format(d), end='\r')
    criteria = new_df.loc[:,d] == 'Y'
    if len(new_df[criteria].index) == 200:
        daily_k200.loc[:199,d] = list(new_df[criteria].index)
    elif len(new_df[criteria].index) == 199:
        daily_k200.loc[:198,d] = list(new_df[criteria].index)
    elif len(new_df[criteria].index) == 201:
        daily_k200.loc[:200,d] = list(new_df[criteria].index)
    elif len(new_df[criteria].index) == 202:
        daily_k200.loc[:201,d] = list(new_df[criteria].index)

In [30]:
# 결과값
daily_k200

,20100101,20100102,20100103,20100104,20100105,20100106,20100107,20100108,20100109,20100110,...,20191222,20191223,20191224,20191225,20191226,20191227,20191228,20191229,20191230,20191231
0,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,...,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스
1,경방,경방,경방,경방,경방,경방,경방,경방,경방,경방,...,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로
2,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,...,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행
3,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,...,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운
4,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,...,두산,두산,두산,두산,두산,두산,두산,두산,두산,두산
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,...,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발
198,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,...,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업
199,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,...,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주
200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# NaN값을 빈칸으로 수정
for d in lv2_index:
    daily_k200.loc[:, d] = daily_k200.loc[:, d].fillna('')
daily_k200

,20100101,20100102,20100103,20100104,20100105,20100106,20100107,20100108,20100109,20100110,...,20191222,20191223,20191224,20191225,20191226,20191227,20191228,20191229,20191230,20191231
0,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,...,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스
1,경방,경방,경방,경방,경방,경방,경방,경방,경방,경방,...,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로
2,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,...,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행
3,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,...,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운
4,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,...,두산,두산,두산,두산,두산,두산,두산,두산,두산,두산
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,...,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발
198,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,...,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업
199,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,...,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주
200,,,,,,,,,,,...,,,,,,,,,,


In [204]:
daily_k200.to_csv(r'C:\Users\PC\OneDrive\Data Science\Weak-supervision\생존편향 제거\완료된 데이터\daily_k200.csv', encoding='euc-kr')

### 20100101 ~ 20191231의 기간동안 한 번이라도 KOSPI200에 편입된 적이 있는 기업 리스트를 구하기 위한 과정

In [32]:
daily_k200_copy = daily_k200.copy()
daily_k200_copy

,20100101,20100102,20100103,20100104,20100105,20100106,20100107,20100108,20100109,20100110,...,20191222,20191223,20191224,20191225,20191226,20191227,20191228,20191229,20191230,20191231
0,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,...,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스
1,경방,경방,경방,경방,경방,경방,경방,경방,경방,경방,...,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로
2,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,...,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행
3,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,...,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운
4,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,...,두산,두산,두산,두산,두산,두산,두산,두산,두산,두산
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,...,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발
198,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,...,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업
199,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,...,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주
200,,,,,,,,,,,...,,,,,,,,,,


In [33]:
# all_list에 20100101 ~ 20191231 기간동안의 모든 KOSPI200 리스트를 담음 (202 * 3,652 = 737,704)
all_list = []
for d in lv2_index:
    all_list = all_list + list(daily_k200_copy.loc[:, d])
all_list

['동화약품',
 '경방',
 '삼양홀딩스',
 '유한양행',
 'CJ대한통운',
 '하이트진로홀딩스',
 '두산',
 '대림산업',
 '일동홀딩스',
 '한국테크놀로지그룹',
 '기아차',
 '조선내화',
 '동아쏘시오홀딩스',
 'SK하이닉스',
 '영풍',
 '유수홀딩스',
 '현대건설',
 '삼성화재',
 '삼성물산',
 '한화',
 'DB하이텍',
 'CJ',
 'JW중외제약',
 'LG상사',
 '대한제분',
 '금호전기',
 '동국제강',
 '제일모직',
 '세아베스틸',
 '대한전선',
 '동양',
 '종근당홀딩스',
 '대상',
 'SK네트웍스',
 '대한제당',
 '오리온홀딩스',
 'KISCO홀딩스',
 '코오롱',
 '아세아',
 '고려제강',
 '롯데푸드',
 '한국제지',
 '넥센타이어',
 'KCC',
 '한독',
 '아모레G',
 '금호산업',
 '부광약품',
 '세아제강지주',
 '일성신약',
 '태광산업',
 '한일홀딩스',
 '쌍용양회',
 '현대증권',
 '한진중공업홀딩스',
 '대한항공',
 '영진약품',
 'LG',
 'S&T중공업',
 'SK',
 '유니온스틸',
 '남양유업',
 '삼양제넥스',
 '롯데정밀화학',
 '현대제철',
 '대덕GDS',
 '한솔홀딩스',
 '신세계',
 '농심',
 '세방전지',
 '한솔테크닉스',
 '효성',
 '외환은행',
 '롯데지주',
 '삼광글라스',
 '빙그레',
 '대구은행',
 '부산은행',
 '롯데칠성',
 '현대차',
 'POSCO',
 '삼진제약',
 '삼영전자',
 '삼성전자',
 'NH투자증권',
 'SK디스커버리',
 'LS',
 '녹십자',
 'GS건설',
 '카프로',
 '한일현대시멘트',
 '삼성SDI',
 '대한유화',
 '미래에셋대우',
 'AK홀딩스',
 '오뚜기',
 '디티알오토모티브',
 '일양약품',
 '도레이케미칼',
 '대덕전자',
 '한국기술산업',
 '율촌화학',
 '한미사이언스',
 '삼성전기',
 '무림페이퍼',
 '광동제약',
 'KC그

In [34]:
len(all_list)

737704

In [35]:
# set자료형으로 바꿔줌에 따라 빈칸으로 바꿔줬던 NaN값들은 모두 하나의 값 ''로 바뀌고, 중복된 이름 모두 제거됨
# 빈칸이었던 ''를 제거하면 KOSPI200에 한번이라도 편입됐던 기업의 총 개수는 311개
lv2_companies = list(set(all_list))
lv2_companies   # 312개

['',
 '대한제분',
 '유니드',
 '남해화학',
 '동아에스티',
 '벽산',
 'CJ제일제당',
 'SKC',
 '우리금융지주',
 '삼양홀딩스',
 '코오롱인더',
 '파미셀',
 '한솔홀딩스',
 'DB하이텍',
 'CJ',
 '팬오션',
 '태광산업',
 'SBS',
 '한미반도체',
 '녹십자홀딩스',
 '한국단자',
 '도레이케미칼',
 '한국콜마',
 '퍼시스',
 '롯데정밀화학',
 'LIG넥스원',
 '오리온',
 'STX엔진',
 '삼성SDI',
 'KB금융',
 'KISCO홀딩스',
 '아모레G',
 '무림P&P',
 '보령제약',
 '동국제강',
 '삼영전자',
 '한독',
 '동서',
 '우리금융',
 '이수화학',
 '알앤엘바이오',
 '동양',
 '현대글로비스',
 '한국철강',
 '동부제철',
 'S&T모티브',
 '한국전력',
 'GS리테일',
 '금호석유',
 '만도',
 '동화약품',
 '하이트맥주',
 '휠라홀딩스',
 '휴켐스',
 '락앤락',
 '한솔케미칼',
 '현대홈쇼핑',
 '호텔신라',
 '전기초자',
 'SK네트웍스',
 '녹십자',
 '자화전자',
 '코스맥스',
 '삼성에스디에스',
 '삼진제약',
 '영풍',
 '조광피혁',
 '대덕전자',
 '유한양행',
 '제일모직',
 '포스코인터내셔널',
 '롯데하이마트',
 '세하',
 'GKL',
 'POSCO',
 '한일홀딩스',
 '현대차',
 '부광약품',
 '에이블씨엔씨',
 '에스원',
 '유니온스틸',
 '신한지주',
 '종근당홀딩스',
 'STX중공업',
 '동아쏘시오홀딩스',
 '현대미포조선',
 '대한유화',
 '한국쉘석유',
 '고려제강',
 'LG이노텍',
 '일진머티리얼즈',
 '두산인프라코어',
 '대한제당',
 'LG하우시스',
 '대우건설',
 '현대그린푸드',
 '삼성중공업',
 '율촌화학',
 '롯데케미칼',
 '부산은행',
 '한국항공우주',
 '카카오',
 '동일산업',
 'DB손해보험',
 '제일파마홀딩스',
 '세종공업

In [36]:
del lv2_companies[0]
lv2_companies = sorted(lv2_companies)   # 순서대로 정렬
lv2_companies   # 311개

['AK홀딩스',
 'BGF',
 'BGF리테일',
 'BNK금융지주',
 'CJ',
 'CJ CGV',
 'CJ대한통운',
 'CJ제일제당',
 'DB손해보험',
 'DB하이텍',
 'DGB금융지주',
 'GKL',
 'GS',
 'GS건설',
 'GS리테일',
 'HDC',
 'HDC현대산업개발',
 'HSD엔진',
 'JW중외제약',
 'JW홀딩스',
 'KB금융',
 'KCC',
 'KC그린홀딩스',
 'KISCO홀딩스',
 'KPX케미칼',
 'KT',
 'KT&G',
 'LF',
 'LG',
 'LG디스플레이',
 'LG상사',
 'LG생명과학',
 'LG생활건강',
 'LG유플러스',
 'LG이노텍',
 'LG전자',
 'LG하우시스',
 'LG화학',
 'LIG넥스원',
 'LS',
 'LS산전',
 'NAVER',
 'NH투자증권',
 'OCI',
 'POSCO',
 'S&TC',
 'S&T모티브',
 'S&T중공업',
 'S-Oil',
 'SBS',
 'SK',
 'SKC',
 'SK네트웍스',
 'SK디스커버리',
 'SK이노베이션',
 'SK케미칼',
 'SK텔레콤',
 'SK하이닉스',
 'SPC삼립',
 'STX',
 'STX엔진',
 'STX조선해양',
 'STX중공업',
 'TBH글로벌',
 '강원랜드',
 '경방',
 '고려아연',
 '고려제강',
 '광동제약',
 '광전자',
 '국도화학',
 '금호산업',
 '금호석유',
 '금호전기',
 '금호타이어',
 '기아차',
 '기업은행',
 '남양유업',
 '남해화학',
 '넥센타이어',
 '넷마블',
 '녹십자',
 '녹십자홀딩스',
 '농심',
 '대교',
 '대구은행',
 '대덕GDS',
 '대덕전자',
 '대림산업',
 '대상',
 '대우건설',
 '대우조선해양',
 '대웅제약',
 '대원강업',
 '대한유화',
 '대한전선',
 '대한제강',
 '대한제당',
 '대한제분',
 '대한항공',
 '도레이케미칼',
 '동국제강',
 '동부제철',
 '동서',
 '동아쏘시오홀딩스

여기서 '우리금융지주'를 제거해줘야 하는데, 그 이유는 다음과 같다.
- Data Guide 상에서 '우리금융'(기타)(053000) 회사의 기업코드를 넣고 Dart api를 돌린 결과 Dart 상에서의 이름은 '우리금융'이 아닌 '우리금융지주'라는 걸 알 수 있었다.(심지어 Dart api 상 '우리금융지주'의 기업코드는 375302로 다름(Dart api를 사용하다보니 input 기업코드와 output 기업코드의 결과가 다른 경우가 종종 있었음)) 
- 또한, Data Guide 상 '우리금융지주'(유가)(316140)의 기업코드를 넣고 Dart api를 돌리면 아무런 사업보고서가 나오지 않는다. Dart 기업공시 사이트에서 확인해봐도 '우리금융지주'(유가)(316140)의 사업보고서가 없고, '우리금융'(기타)(053000)의 사업보고서만 있는 것을 알 수 있었다.
- 결론 : 위와 같은 이유와, 네이버금융 & Dart를 비교 분석해본 결과 Data Guide 상의 '우리금융'과 Dart 상에서 사업보고서를 낸 '우리금융지주'는 같은 기업이라는 것을 알 수 있었다. 따라서, 혼란을 제거하기 위해 기존의 '우리금융'의 이름을 '우리금융지주'로 바꾸면서 기존의 '우리금융지주'는 삭제하였다.

In [37]:
lv2_companies.index('우리금융지주')

194

In [38]:
del lv2_companies[194]

In [42]:
len(lv2_companies)   # 따라서, 전체 기업의 개수는 310개

310

## date_df

date_df를 만드는 이유는 lv2_df(index = 20100101 ~ 20191231, columns = lv2_companies)의 형태로 봤을 때, 
각 기업별로 KOSPI200에 편입됐던 날짜에는 O, 편입되지 않았던 날짜에는 X로 채워넣어 한 눈에 살펴보기 위함

In [39]:
# lv2_index는 20100101~20191231 전체 기간, lv2_companies는 생존편향을 제거한 KOSPI200 기업 리스트
date_df = pd.DataFrame(np.full(shape = (len(lv2_index), len(lv2_companies)), fill_value=np.nan), index = lv2_index, columns = lv2_companies)
date_df

,AK홀딩스,BGF,BGF리테일,BNK금융지주,CJ,CJ CGV,CJ대한통운,CJ제일제당,DB손해보험,DB하이텍,...,현대해상,현대홈쇼핑,호텔신라,화신,효성,효성중공업,후성,휠라홀딩스,휴비스,휴켐스
20100101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20100102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20100103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20100104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20100105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20191228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20191229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20191230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
daily_k200_copy

,20100101,20100102,20100103,20100104,20100105,20100106,20100107,20100108,20100109,20100110,...,20191222,20191223,20191224,20191225,20191226,20191227,20191228,20191229,20191230,20191231
0,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,동화약품,...,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스
1,경방,경방,경방,경방,경방,경방,경방,경방,경방,경방,...,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로,하이트진로
2,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,삼양홀딩스,...,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행
3,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,유한양행,...,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운
4,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,CJ대한통운,...,두산,두산,두산,두산,두산,두산,두산,두산,두산,두산
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,한진중공업,...,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발,HDC현대산업개발
198,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,CJ제일제당,...,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업,효성중공업
199,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,KB금융,...,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주,우리금융지주
200,,,,,,,,,,,...,,,,,,,,,,


In [43]:
for seq, corp in enumerate(lv2_companies):
    print("Process : {} | Total : {}".format(seq+1, len(lv2_companies)), end='\r')
    for d in lv2_index:
        if (corp in daily_k200_copy.loc[:,d].values) == True:   # 일별(d)로 해당 기업(corp)이 KOSPI200 리스트에 있다면
            date_df.loc[d,corp] = 'O'                           # 해당 날짜(d)에 'O' 처리
        else:
            date_df.loc[d,corp] = 'X'                           # 없다면 'X' 처리

In [44]:
date_df

,AK홀딩스,BGF,BGF리테일,BNK금융지주,CJ,CJ CGV,CJ대한통운,CJ제일제당,DB손해보험,DB하이텍,...,현대해상,현대홈쇼핑,호텔신라,화신,효성,효성중공업,후성,휠라홀딩스,휴비스,휴켐스
20100101,O,X,X,X,O,X,O,O,X,O,...,X,X,X,X,O,X,O,X,X,O
20100102,O,X,X,X,O,X,O,O,X,O,...,X,X,X,X,O,X,O,X,X,O
20100103,O,X,X,X,O,X,O,O,X,O,...,X,X,X,X,O,X,O,X,X,O
20100104,O,X,X,X,O,X,O,O,X,O,...,X,X,X,X,O,X,O,X,X,O
20100105,O,X,X,X,O,X,O,O,X,O,...,X,X,X,X,O,X,O,X,X,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191227,O,O,O,O,O,O,O,O,O,O,...,O,O,O,X,O,O,O,O,X,O
20191228,O,O,O,O,O,O,O,O,O,O,...,O,O,O,X,O,O,O,O,X,O
20191229,O,O,O,O,O,O,O,O,O,O,...,O,O,O,X,O,O,O,O,X,O
20191230,O,O,O,O,O,O,O,O,O,O,...,O,O,O,X,O,O,O,O,X,O


In [45]:
# 총 23개의 기업이름이 Data Guide & Dart 상에서 다르기 때문에, 통일을 위해 Data-Guide의 이름을 Dart 상의 이름으로 변경시켜 줌
# 기업코드로 비교해서 같은 기업들임을 확인 (ex.'케이씨오에너지'(011400)->'아진카인텍'(011400))
date_df.rename(columns = {"KCC":"케이씨씨", "KT":"케이티", "KT&G":"케이티앤지", "POSCO":"포스코", 
                          "TBH글로벌":"티비에이치글로벌", "금호석유":"금호석유화학", "기아차":"기아자동차", 
                          "롯데칠성":"롯데칠성음료", "삼성화재":"삼성화재해상보험", "삼영전자":"삼영전자공업", 
                          "쌍용양회":"쌍용양회공업", "쌍용차":"쌍용자동차", "아모레G":"아모레퍼시픽그룹", 
                          "알앤엘바이오":"알바이오", "외환은행":"하나은행", "우리금융":"우리금융지주", 
                          "전기초자":"한국전기초자", "케이씨오에너지":"아진카인텍", "한국단자":"한국단자공업", 
                          "한국전력":"한국전력공사", "현대엘리베이":"현대엘리베이터", "현대증권":"케이비증권", 
                          "현대차":"현대자동차"}, inplace=True)

In [267]:
date_df.to_csv(r'C:\Users\PC\OneDrive\Data Science\Weak-supervision\생존편향 제거\완료된 데이터\date_df.csv', encoding='euc-kr')

In [48]:
date_df = pd.read_csv(r'C:\Users\PC\OneDrive\Data Science\Weak-supervision\생존편향 제거\완료된 데이터\date_df.csv', encoding='euc-kr', index_col=0)
# lv2_companies = list(date_df.columns)
# lv2_companies
date_df

,AK홀딩스,BGF,BGF리테일,BNK금융지주,CJ,CJ CGV,CJ대한통운,CJ제일제당,DB손해보험,DB하이텍,...,현대해상,현대홈쇼핑,호텔신라,화신,효성,효성중공업,후성,휠라홀딩스,휴비스,휴켐스
Date,,,,,,,,,,,,,,,,,,,,,
20100101,O,X,X,X,O,X,O,O,X,O,...,X,X,X,X,O,X,O,X,X,O
20100102,O,X,X,X,O,X,O,O,X,O,...,X,X,X,X,O,X,O,X,X,O
20100103,O,X,X,X,O,X,O,O,X,O,...,X,X,X,X,O,X,O,X,X,O
20100104,O,X,X,X,O,X,O,O,X,O,...,X,X,X,X,O,X,O,X,X,O
20100105,O,X,X,X,O,X,O,O,X,O,...,X,X,X,X,O,X,O,X,X,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191227,O,O,O,O,O,O,O,O,O,O,...,O,O,O,X,O,O,O,O,X,O
20191228,O,O,O,O,O,O,O,O,O,O,...,O,O,O,X,O,O,O,O,X,O
20191229,O,O,O,O,O,O,O,O,O,O,...,O,O,O,X,O,O,O,O,X,O


In [53]:
date_df.columns

Index(['AK홀딩스', 'BGF', 'BGF리테일', 'BNK금융지주', 'CJ', 'CJ CGV', 'CJ대한통운', 'CJ제일제당',
       'DB손해보험', 'DB하이텍',
       ...
       '현대해상', '현대홈쇼핑', '호텔신라', '화신', '효성', '효성중공업', '후성', '휠라홀딩스', '휴비스',
       '휴켐스'],
      dtype='object', length=310)

In [54]:
date_df.index

Int64Index([20100101, 20100102, 20100103, 20100104, 20100105, 20100106,
            20100107, 20100108, 20100109, 20100110,
            ...
            20191222, 20191223, 20191224, 20191225, 20191226, 20191227,
            20191228, 20191229, 20191230, 20191231],
           dtype='int64', name='Date', length=3652)

In [55]:
date_df.index = list(map(str, date_df.index))

In [56]:
symbol_dic

{'조흥은행': '000010',
 '조흥은행(1신)': '000011',
 '조흥은행(2신)': '000012',
 '동화약품': '000020',
 '동화약품공업(1신)': '000021',
 '동화약품(전환1신)': '000022',
 '동화약품(전환2신)': '000023',
 '우리은행': '000030',
 '한빛은행(1신)': '000031',
 '한국상업은행(2신)': '000032',
 'KR모터스': '000040',
 '효성기계공업(1신)': '000041',
 '효성기계공업(2신)': '000042',
 '경방': '000050',
 '메리츠화재': '000060',
 '동양화재(1신)': '000061',
 '동양화재(2신)': '000062',
 '삼양홀딩스': '000070',
 '삼양사(1신)': '000071',
 '삼양사(2신)': '000072',
 '삼양사(3신)': '000073',
 '삼양홀딩스우': '000075',
 '삼양사(1우1신)': '000076',
 '하이트진로': '000080',
 '진로(1신)': '000081',
 '진로(2신)': '000082',
 '진로(1우2신)': '000083',
 '진로우': '000085',
 '진로(1우1신)': '000086',
 '하이트진로2우B': '000087',
 '두산상사': '000090',
 '두산상사(1신)': '000091',
 '유한양행': '000100',
 '유한양행(1신)': '000101',
 '유한양행(2신)': '000102',
 '유한양행우': '000105',
 '유한양행(1우1신)': '000106',
 '유한양행2우B': '000107',
 '제일은행': '000110',
 '제일은행(1신)': '000111',
 '제일은행(2신)': '000112',
 'CJ대한통운': '000120',
 '대한통운(1신)': '000121',
 '대한통운(2신)': '000122',
 '한일은행': '000130',
 '한일은행(1신)': '00

In [57]:
# Data Guide에서 받은 자료에 있는 모든 기업 & 기업코드를 토대로 만든 symbol_dic을 이용하여 lv2_companies에 있는 기업들의 기업코드를 symbol에 저장(Dart api를 이용하기 위해)
symbol = []
for corp in lv2_companies:
    symbol.append(symbol_dic[corp])

In [58]:
symbol

['006840',
 '027410',
 '282330',
 '138930',
 '001040',
 '079160',
 '000120',
 '097950',
 '005830',
 '000990',
 '139130',
 '114090',
 '078930',
 '006360',
 '007070',
 '012630',
 '294870',
 '082740',
 '001060',
 '096760',
 '105560',
 '002380',
 '009440',
 '001940',
 '025000',
 '030200',
 '033780',
 '093050',
 '003550',
 '034220',
 '001120',
 '068870',
 '051900',
 '032640',
 '011070',
 '066570',
 '108670',
 '051910',
 '079550',
 '006260',
 '010120',
 '035420',
 '005940',
 '010060',
 '005490',
 '100840',
 '064960',
 '003570',
 '010950',
 '034120',
 '034730',
 '011790',
 '001740',
 '006120',
 '096770',
 '285130',
 '017670',
 '000660',
 '005610',
 '011810',
 '077970',
 '067250',
 '071970',
 '084870',
 '035250',
 '000050',
 '010130',
 '002240',
 '009290',
 '017900',
 '007690',
 '002990',
 '011780',
 '001210',
 '073240',
 '000270',
 '024110',
 '003920',
 '025860',
 '002350',
 '251270',
 '006280',
 '005250',
 '004370',
 '019680',
 '005270',
 '004130',
 '008060',
 '000210',
 '001680',
 '047040',

In [59]:
len(symbol)

310

In [356]:
# 주소(url) setting

API_KEY = "9a9929b43fd4c79e06cc04d50eb34a70a0a576b1"  # Dart에서 발급받은 API key
start_date = "20100101"
end_date = "20191231"
page_set = "100"   # 페이지당 건수
# auth : 발급받은 인증키(40자리)
# crp_cd : 공시대상회사의 종목코드(상장사:6자리 숫자) 또는 고유번호(기타법인:8자리 숫자)
# start_dt, end_dt : 검색시작, 끝 접수일자
# dsn_tp : A001(사업보고서)

url_list = []
for sym in symbol:
    company_code = sym   
    url = "http://dart.fss.or.kr/api/search.xml?auth=" + API_KEY + "&crp_cd=" + company_code + "&start_dt=" + start_date + "&end_dt=" + end_date + "&bsn_tp=A001&page_set=" + page_set  
    url_list.append(url)
url_list

['http://dart.fss.or.kr/api/search.xml?auth=9a9929b43fd4c79e06cc04d50eb34a70a0a576b1&crp_cd=006840&start_dt=20100101&end_dt=20191231&bsn_tp=A001&page_set=100',
 'http://dart.fss.or.kr/api/search.xml?auth=9a9929b43fd4c79e06cc04d50eb34a70a0a576b1&crp_cd=027410&start_dt=20100101&end_dt=20191231&bsn_tp=A001&page_set=100',
 'http://dart.fss.or.kr/api/search.xml?auth=9a9929b43fd4c79e06cc04d50eb34a70a0a576b1&crp_cd=282330&start_dt=20100101&end_dt=20191231&bsn_tp=A001&page_set=100',
 'http://dart.fss.or.kr/api/search.xml?auth=9a9929b43fd4c79e06cc04d50eb34a70a0a576b1&crp_cd=138930&start_dt=20100101&end_dt=20191231&bsn_tp=A001&page_set=100',
 'http://dart.fss.or.kr/api/search.xml?auth=9a9929b43fd4c79e06cc04d50eb34a70a0a576b1&crp_cd=001040&start_dt=20100101&end_dt=20191231&bsn_tp=A001&page_set=100',
 'http://dart.fss.or.kr/api/search.xml?auth=9a9929b43fd4c79e06cc04d50eb34a70a0a576b1&crp_cd=079160&start_dt=20100101&end_dt=20191231&bsn_tp=A001&page_set=100',
 'http://dart.fss.or.kr/api/search.xml?a

In [357]:
len(url_list)

310

Dart 기업공시 사이트를 보면, 각 기업들은 제일 처음에 제출한 사업보고서에 수정사항이 있으면 [기재정정]사업보고서 혹은 [첨부정정]사업보고서를 제출한다. 제일 최신으로 수정된 사업보고서를 이용하고 싶다면 Dart api의 결과값 중 사업보고서의 유형을 나타내는 'rmk'가 '정' 혹은 '정연'인 것들을 제외해야한다. 하지만 처음 제출한 보고서와 수정된 보고서를 비교해봤을 때, 텍스트 내용 자체의 차이가 거의 없고, filing 날짜가 중요하기 때문에 첫 보고서가 나온 시점을 사용하기로 한다.

추가로, Dart api를 사용하여 짧은 시간동안 한번에 100번 이상 Dart에 접근하면 약 하루동안 IP가 차단된다. 따라서 Dart api를 이용하여 사업보고서들의 목록을 가져올 때 1초의 time.sleep을 걸어준다.

In [358]:
# KOSPI200에 한번이라도 포함됐던 회사들의 2010 ~ 2019년 모든 사업보고서를 데이터프레임으로 반환

def data_extract():
    url_lists = url_list
    data_finals = pd.DataFrame()   #
    for url in url_lists:
        time.sleep(1)       # IP차단을 예방하기 위해 1초의 time.sleep을 걸어줌
        xmlsoup = BeautifulSoup(urlopen(url).read(), 'html.parser')   # Dart '회사별검색'에 해당 회사를 검색했을 때 나오는 사이트
        te = xmlsoup.findAll("list")    # 'list'는 각 사업보고서를 담고 있는 HTML코드이므로 te에는 해당 회사의 모든 사업보고서가 담김
        data = pd.DataFrame()   ##
        for t in te:
            # Dart api의 결과값들이 데이터프레임으로 담김
            temp = pd.DataFrame(([[t.crp_cls.string, t.crp_nm.string, t.crp_cd.string, t.rpt_nm.string,
                              t.rcp_no.string, t.flr_nm.string, t.rcp_dt.string, t.rmk.string]]),
                              columns = ["crp_cls", "crp_nm", "crp_cd", "rpt_nm", "rcp_no", "flr_nm", "rcp_dt", "rmk"])
            data = pd.concat([data,temp])   ## 한 루프를 돌 때마다 개별 사업보고서의 dataframe을 concat해줌
            data_result = data
#         data_result = data_result[data_result['rmk'] != '정연']   # '정연'은 수정된 사업보고서가 제출되기 이전의 보고서
#         data_result = data_result[data_result['rmk'] != '정']     # '정'은 수정된 사업보고서가 제출되기 이전의 보고서
        data_finals = pd.concat([data_finals, data_result])    # 한 루프를 돌 때마다 회사별 사업보고서의 전체 dataframe을 concat해줌
        data_finals = data_finals.reset_index(drop=True)
        
        # 중복된 사업보고서 삭제
        data_finals = data_finals.drop_duplicates(['crp_cls', 'crp_nm', 'crp_cd', 'rpt_nm', 'rcp_no', 'flr_nm', 'rcp_dt', 'rmk'], keep='first')
    return data_finals

In [359]:
report_result = data_extract()
report_result

,crp_cls,crp_nm,crp_cd,rpt_nm,rcp_no,flr_nm,rcp_dt,rmk
0,Y,AK홀딩스,006840,사업보고서 (2018.12),20190401004433,AK홀딩스,20190401,연
1,Y,AK홀딩스,006840,사업보고서 (2017.12),20180330002417,AK홀딩스,20180330,연
2,Y,AK홀딩스,006840,사업보고서 (2016.12),20170331003282,AK홀딩스,20170331,연
3,Y,AK홀딩스,006840,사업보고서 (2015.12),20160330003602,AK홀딩스,20160330,연
4,Y,AK홀딩스,006840,[기재정정]사업보고서 (2014.12),20150605000083,AK홀딩스,20150605,연
...,...,...,...,...,...,...,...,...
3685,Y,휴켐스,069260,[기재정정]사업보고서 (2012.12),20130508000239,휴켐스,20130508,연
3686,Y,휴켐스,069260,사업보고서 (2012.12),20130401000720,휴켐스,20130401,정연
3687,Y,휴켐스,069260,사업보고서 (2011.12),20120330000492,휴켐스,20120330,연
3688,Y,휴켐스,069260,[첨부추가]사업보고서 (2010.12),20110331002418,휴켐스,20110331,연


In [360]:
# csv file로 저장
report_result.to_csv(r'C:\Users\PC\OneDrive\Data Science\Weak-supervision\생존편향 제거\All_Report_New.csv', encoding='euc-kr') # 한글깨짐 방지

In [64]:
dart_api_result = pd.read_csv(r'C:\Users\PC\OneDrive\Data Science\Weak-supervision\생존편향 제거\완료된 데이터\All_Report_New.csv', encoding='euc-kr', index_col=0)

In [65]:
dart_api_result

,crp_cls,crp_nm,crp_cd,rpt_nm,rcp_no,flr_nm,rcp_dt,rmk
0,Y,AK홀딩스,6840,사업보고서 (2018.12),20190401004433,AK홀딩스,20190401,연
1,Y,AK홀딩스,6840,사업보고서 (2017.12),20180330002417,AK홀딩스,20180330,연
2,Y,AK홀딩스,6840,사업보고서 (2016.12),20170331003282,AK홀딩스,20170331,연
3,Y,AK홀딩스,6840,사업보고서 (2015.12),20160330003602,AK홀딩스,20160330,연
4,Y,AK홀딩스,6840,[기재정정]사업보고서 (2014.12),20150605000083,AK홀딩스,20150605,연
...,...,...,...,...,...,...,...,...
3685,Y,휴켐스,69260,[기재정정]사업보고서 (2012.12),20130508000239,휴켐스,20130508,연
3686,Y,휴켐스,69260,사업보고서 (2012.12),20130401000720,휴켐스,20130401,정연
3687,Y,휴켐스,69260,사업보고서 (2011.12),20120330000492,휴켐스,20120330,연
3688,Y,휴켐스,69260,[첨부추가]사업보고서 (2010.12),20110331002418,휴켐스,20110331,연


In [ ]:
# difference name in Dart api & Data-Guide(found by corporate code)
# change Data-Guide name to Dart name
dart_name = list(dart_api_result.loc[:, 'crp_nm'])
dg_name = lv2_companies

for name in dg_name:
    if (name in dart_name) == False:
        print(name)   # 24개

In [377]:
# Data-Guide 상 '알루코(021570)'는 20161228~20170914의 기간동안 KOSPI200에 편입돼 있었지만, Dart에 사업보고서가 전혀 없고 네이버금융에도 검색 안되므로 제거.
# del date_df['알루코']
# del date_df['우리금융지주']
date_df.rename(columns = {"KCC":"케이씨씨", "KT":"케이티", "KT&G":"케이티앤지", "POSCO":"포스코", "TBH글로벌":"티비에이치글로벌", 
                          "금호석유":"금호석유화학", "기아차":"기아자동차", "롯데칠성":"롯데칠성음료", "삼성화재":"삼성화재해상보험", 
                          "삼영전자":"삼영전자공업", "쌍용양회":"쌍용양회공업", "쌍용차":"쌍용자동차", "아모레G":"아모레퍼시픽그룹", 
                          "알앤엘바이오":"알바이오", "외환은행":"하나은행", "우리금융":"우리금융지주", "전기초자":"한국전기초자", 
                          "케이씨오에너지":"아진카인텍", "한국단자":"한국단자공업", "한국전력":"한국전력공사", 
                          "현대엘리베이":"현대엘리베이터", "현대증권":"케이비증권", "현대차":"현대자동차"}, inplace=True)

In [276]:
lv2_companies = list(date_df.columns.values)

In [336]:
date_df[date_df.loc[:,'애경산업']=='O'].index

Int64Index([20190614, 20190615, 20190616, 20190617, 20190618, 20190619,
            20190620, 20190621, 20190622, 20190623,
            ...
            20191222, 20191223, 20191224, 20191225, 20191226, 20191227,
            20191228, 20191229, 20191230, 20191231],
           dtype='int64', name='Unnamed: 0', length=201)

In [337]:
qwe = pd.date_range(start='20190614', end='20191231')
len(qwe)

201

search_corp 포함된 기업들 : 'GS리테일', 'KB금융', 'LG상사', 'STX조선해양', '금호전기', '기아자동차', '대림산업',
       '대한제분', '동아쏘시오홀딩스', '빙그레', '삼성SDI', '서연', '서흥', '세아제강지주', '에스엘',
       '엔씨소프트', '영원무역', '우리금융지주', '일신방직', '코스맥스비티아이', '태광산업', '하나금융지주',
       '한국가스공사', '한국조선해양', '한솔케미칼', '한솔테크닉스', '한전기술', '한진중공업', '한화',
       '한화생명', '현대글로비스', '현대모비스', '현대해상', '후성', '삼성물산'

In [508]:
search_corp = ['007070', '105560', '001120', '067250', '001210', '000270', '000210', '001130', '000640', '005180', 
               '006400', '007860', '008490', '003030', '005850', '036570', '111770', '053000', '003200', '044820', 
               '003240', '086790', '036460', '009540', '014680', '004710', '052690', '097230', '000880', '088350', 
               '086280', '012330', '001450', '093370', '000830']

In [509]:
# 주소(url) setting

API_KEY = "9a9929b43fd4c79e06cc04d50eb34a70a0a576b1"
start_date = "20100101"
end_date = "20191231"
page_set = "100"   # 페이지당 건수
# auth : 발급받은 인증키(40자리)
# crp_cd : 공시대상회사의 종목코드(상장사:6자리 숫자) 또는 고유번호(기타법인:8자리 숫자)
# start_dt, end_dt : 검색시작,끝 접수일자
# dsn_tp : A001(사업보고서)

url_list = []
for sym in search_corp:
    company_code = sym   
    url = "http://dart.fss.or.kr/api/search.xml?auth=" + API_KEY + "&crp_cd=" + company_code + "&start_dt=" + start_date + "&end_dt=" + end_date + "&bsn_tp=A001&page_set=" + page_set  
    url_list.append(url)
url_list

['http://dart.fss.or.kr/api/search.xml?auth=9a9929b43fd4c79e06cc04d50eb34a70a0a576b1&crp_cd=007070&start_dt=20100101&end_dt=20191231&bsn_tp=A001&page_set=100',
 'http://dart.fss.or.kr/api/search.xml?auth=9a9929b43fd4c79e06cc04d50eb34a70a0a576b1&crp_cd=105560&start_dt=20100101&end_dt=20191231&bsn_tp=A001&page_set=100',
 'http://dart.fss.or.kr/api/search.xml?auth=9a9929b43fd4c79e06cc04d50eb34a70a0a576b1&crp_cd=001120&start_dt=20100101&end_dt=20191231&bsn_tp=A001&page_set=100',
 'http://dart.fss.or.kr/api/search.xml?auth=9a9929b43fd4c79e06cc04d50eb34a70a0a576b1&crp_cd=067250&start_dt=20100101&end_dt=20191231&bsn_tp=A001&page_set=100',
 'http://dart.fss.or.kr/api/search.xml?auth=9a9929b43fd4c79e06cc04d50eb34a70a0a576b1&crp_cd=001210&start_dt=20100101&end_dt=20191231&bsn_tp=A001&page_set=100',
 'http://dart.fss.or.kr/api/search.xml?auth=9a9929b43fd4c79e06cc04d50eb34a70a0a576b1&crp_cd=000270&start_dt=20100101&end_dt=20191231&bsn_tp=A001&page_set=100',
 'http://dart.fss.or.kr/api/search.xml?a

In [510]:
len(url_list)

35

In [476]:
# All_Report_Final 저장한 것에서 수정사항([첨부정정]인 경우 '정연'보고서를 가져오고, 가장 마지막 기재정정 사업보고서 가져오기)

def data_extract_2():
    url_lists = url_list
    data_finals = pd.DataFrame()   #
    for url in url_lists:
        time.sleep(1)       # IP차단을 예방하기 위해
        xmlsoup = BeautifulSoup(urlopen(url).read(), 'html.parser')
        te = xmlsoup.findAll("list")
        data = pd.DataFrame()   ##
        for t in te:
            temp = pd.DataFrame(([[t.crp_cls.string, t.crp_nm.string, t.crp_cd.string, t.rpt_nm.string,
                              t.rcp_no.string, t.flr_nm.string, t.rcp_dt.string, t.rmk.string]]),
                              columns = ["crp_cls", "crp_nm", "crp_cd", "rpt_nm", "rcp_no", "flr_nm", "rcp_dt", "rmk"])
            data = pd.concat([data,temp])   ##
            data_result = data
#         data_result = data_result[data_result['rmk'] != '정연']   # '정연'은 수정된 사업보고서가 제출되기 이전의 보고서이므로 제외
#         data_result = data_result[data_result['rmk'] != '정']     # '정'은 수정된 사업보고서가 제출되기 이전의 보고서이므로 제외
        data_finals = pd.concat([data_finals, data_result])    ##
        data_finals = data_finals.reset_index(drop=True)
        # 중복된 사업보고서 삭제
        data_finals = data_finals.drop_duplicates(['crp_cls', 'crp_nm', 'crp_cd', 'rpt_nm', 'rcp_no', 'flr_nm', 'rcp_dt', 'rmk'], keep='first')
    return data_finals

In [511]:
search_result = data_extract_2()
search_result

,crp_cls,crp_nm,crp_cd,rpt_nm,rcp_no,flr_nm,rcp_dt,rmk
0,Y,GS리테일,007070,[기재정정]사업보고서 (2018.12),20190401003762,GS리테일,20190401,연
1,Y,GS리테일,007070,사업보고서 (2018.12),20190401001023,GS리테일,20190401,정연
2,Y,GS리테일,007070,사업보고서 (2017.12),20180330002366,GS리테일,20180330,연
3,Y,GS리테일,007070,사업보고서 (2016.12),20170330001704,GS리테일,20170330,연
4,Y,GS리테일,007070,[첨부정정]사업보고서 (2015.12),20160330000435,GS리테일,20160330,연
...,...,...,...,...,...,...,...,...
463,E,삼성물산,00126229,[기재정정]사업보고서 (2012.12),20130403001646,삼성물산,20130403,연
464,E,삼성물산,00126229,사업보고서 (2012.12),20130401002780,삼성물산,20130401,정연
465,E,삼성물산,00126229,[첨부추가]사업보고서 (2011.12),20120330003355,삼성물산,20120330,연
466,E,삼성물산,00126229,[첨부추가]사업보고서 (2010.12),20110331003024,삼성물산,20110331,연


In [515]:
search_result.crp_nm.unique()

array(['GS리테일', 'KB금융', 'LG상사', 'STX조선해양', '금호전기', '기아자동차', '대림산업',
       '대한제분', '동아쏘시오홀딩스', '빙그레', '삼성SDI', '서연', '서흥', '세아제강지주', '에스엘',
       '엔씨소프트', '영원무역', '우리금융지주', '일신방직', '코스맥스비티아이', '태광산업', '하나금융지주',
       '한국가스공사', '한국조선해양', '한솔케미칼', '한솔테크닉스', '한전기술', '한진중공업', '한화',
       '한화생명', '현대글로비스', '현대모비스', '현대해상', '후성', '삼성물산'], dtype=object)

In [516]:
# csv file로 저장
search_result.to_csv(r'C:\Users\PC\OneDrive\Data Science\Weak-supervision\생존편향 제거\Search_Result.csv', encoding='euc-kr') # 한글깨짐 방지

In [68]:
result = pd.read_excel(r'C:\Users\PC\OneDrive\Data Science\Weak-supervision\생존편향 제거\완료된 데이터\All_Report_Removed.xlsx')
result.drop('Unnamed: 0', axis=1, inplace=True)

In [69]:
result

,crp_cls,crp_nm,crp_cd,rpt_nm,rcp_no,flr_nm,rcp_dt,rmk
0,Y,AK홀딩스,6840,사업보고서 (2018.12),20190401004433,AK홀딩스,20190401,연
1,Y,AK홀딩스,6840,사업보고서 (2017.12),20180330002417,AK홀딩스,20180330,연
2,Y,AK홀딩스,6840,사업보고서 (2016.12),20170331003282,AK홀딩스,20170331,연
3,Y,AK홀딩스,6840,사업보고서 (2015.12),20160330003602,AK홀딩스,20160330,연
4,Y,AK홀딩스,6840,사업보고서 (2014.12),20150331003839,AK홀딩스,20150331,정연
...,...,...,...,...,...,...,...,...
2868,Y,휴켐스,69260,사업보고서 (2013.12),20140331003099,휴켐스,20140331,정연
2869,Y,휴켐스,69260,사업보고서 (2012.12),20130401000720,휴켐스,20130401,정연
2870,Y,휴켐스,69260,사업보고서 (2011.12),20120330000492,휴켐스,20120330,연
2871,Y,휴켐스,69260,[첨부추가]사업보고서 (2010.12),20110331002418,휴켐스,20110331,연


In [71]:
yyyymm = result.rpt_nm.map(lambda x : x.split(" ")[-1]).values
year = []
for ym in yyyymm:
    year.append(ym[1:5])

In [72]:
result['연도'] = year
result_copy = result.copy()
result_copy = result.loc[:, ['crp_nm', 'crp_cd', '연도', 'rpt_nm', 'rcp_no', 'rcp_dt', 'rmk']]
result_copy

,crp_nm,crp_cd,연도,rpt_nm,rcp_no,rcp_dt,rmk
0,AK홀딩스,6840,2018,사업보고서 (2018.12),20190401004433,20190401,연
1,AK홀딩스,6840,2017,사업보고서 (2017.12),20180330002417,20180330,연
2,AK홀딩스,6840,2016,사업보고서 (2016.12),20170331003282,20170331,연
3,AK홀딩스,6840,2015,사업보고서 (2015.12),20160330003602,20160330,연
4,AK홀딩스,6840,2014,사업보고서 (2014.12),20150331003839,20150331,정연
...,...,...,...,...,...,...,...
2868,휴켐스,69260,2013,사업보고서 (2013.12),20140331003099,20140331,정연
2869,휴켐스,69260,2012,사업보고서 (2012.12),20130401000720,20130401,정연
2870,휴켐스,69260,2011,사업보고서 (2011.12),20120330000492,20120330,연
2871,휴켐스,69260,2010,[첨부추가]사업보고서 (2010.12),20110331002418,20110331,연


In [73]:
for i in result_copy.rcp_dt.values:
    print(type(i))

<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'nu

<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'nu

In [74]:
result_copy['rcp_dt'] = list(map(str, result_copy.rcp_dt))
result_copy['rcp_no'] = list(map(str, result_copy.rcp_no))

In [81]:
url_lists = []     # 사업보고서 url
for n in result_copy.rcp_no.values:
    url_report = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=" + n
    url_lists.append(url_report)
url_lists

['http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20190401004433',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180330002417',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170331003282',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160330003602',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20150331003839',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20140331003439',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20130401001894',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20120330002633',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20110331001050',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20100331001374',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20190401003382',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180402002991',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170331004623',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160330002688',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20150331003786',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=201904010

In [82]:
len(url_lists)

2873

In [85]:
# 사업보고서 url에 바로 들어가면 각 파트별 HTML코드가 없음.
# 따라서, 각 사업보고서의 HTML 코드에서 '이사의 경영진단 및 분석의견' url_parameters로 url을 구해야 함.

page_list = []     # '이사의 경영진단 및 분석의견' 전체 HTML코드
url_finals = []    # '이사의 경영진단 및 분석의견' url
for process,u in enumerate(url_lists):
    print("Process : {} | Total : {}".format(process+1,len(url_lists)),end='\r')
    time.sleep(1)
    page = BeautifulSoup(urlopen(u).read(), 'html.parser')  
    body = str(page.find('head')).split('이사의 경영진단 및 분석의견",')[1].split('cnt++')[0].split('viewDoc(')[1].split(')')[0].split(', ')
    body = [body[i][1:-1] for i in range(len(body))]   
    url_final = 'http://dart.fss.or.kr/report/viewer.do?rcpNo=' + body[0] + '&dcmNo=' + body[1] + '&eleId=' + body[2] + '&offset=' + body[3] + '&length=' + body[4] + '&dtd=' + body[5]
    time.sleep(1)
    page = BeautifulSoup(urlopen(url_final).read(), 'html.parser')
    page_list.append(page)
    url_finals.append(url_final)

In [86]:
page_list[0]

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html style="border:0">
<head>
<title></title>
<meta content="IE=EmulateIE8" http-equiv="X-UA-Compatible"/>
<meta content="gdi" http-equiv="X-UA-TextLayoutMetrics"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="/css/report_xml.css" rel="stylesheet" type="text/css"/>
</head>
<body bgcolor="#FFFFFF">
<p><br/></p>
<p class="section-1"><a name="toc1">IV. 이사의 경영진단 및 분석의견</a></p>
<p><br/><span style="color:#1600FF;font-weight:bold;">1. 예측정보에 대한 주의사항<br/></span><br/>당사가 동 사업보고서에서 미래에 발생할 것으로 예상,예측한 활동, 사건 또는 현상은 당해 공시서류 작성시점의 사건 및 재무성과에 대하여 회사의 견해를 반영한 것입니다. 동 예측정보는 미래 사업환경과 관련된 다양한 가정에 기초하고 있으며, 동 가정들은 결과적으로 부정확한 것으로 판명될 수도 있습니다. 또한, 이러한 가정들에는<br/>예측정보에서 기재한 예상치와 실제 결과 간에 중요한 차이를 초래할 수 있는 위험, 불확실성 및 기타요인을 포함하고 있습니다. 이러한 중요한 차이를 초래할 수 있는 요인에는 회사 내부경영과 관련된 요인과 외부환경에 관한 요인이 포함되어 있으며, 이에 한하지 않습니다.<br/>당사는 동 예측정보 작성시점 이후에 발생하는 위험 또는 불확실성을 반영하기 위하여 예측정보에 기재한 사

In [87]:
result_copy['url'] = url_finals

In [88]:
result_copy

,crp_nm,crp_cd,연도,rpt_nm,rcp_no,rcp_dt,rmk,url
0,AK홀딩스,6840,2018,사업보고서 (2018.12),20190401004433,20190401,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
1,AK홀딩스,6840,2017,사업보고서 (2017.12),20180330002417,20180330,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
2,AK홀딩스,6840,2016,사업보고서 (2016.12),20170331003282,20170331,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
3,AK홀딩스,6840,2015,사업보고서 (2015.12),20160330003602,20160330,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
4,AK홀딩스,6840,2014,사업보고서 (2014.12),20150331003839,20150331,정연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
...,...,...,...,...,...,...,...,...
2868,휴켐스,69260,2013,사업보고서 (2013.12),20140331003099,20140331,정연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
2869,휴켐스,69260,2012,사업보고서 (2012.12),20130401000720,20130401,정연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
2870,휴켐스,69260,2011,사업보고서 (2011.12),20120330000492,20120330,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
2871,휴켐스,69260,2010,[첨부추가]사업보고서 (2010.12),20110331002418,20110331,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...


In [89]:
def remove_tag(content):             ## 태그를 제외하고 text만 가져오는 함수
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '\n', content)
    return cleantext
def remove_space(content):          # 개행문자인 \n과 \xa0제거
    cleans = re.compile('[\n]|[\xa0]')
    cleantext = re.sub(cleans, '', content)
    return cleantext
def get_text_only(full_text):      # partitioning한 코드에서 재무제표 등의 표를 제거하고 텍스트만 가져오기 위해 'table'태그를 제거
    whole_data = str(full_text).split('<')
    table_data = full_text.find_all('table')
    table_data_li = str(table_data).split('<')
    whole_data_copy = whole_data.copy()

    for table in table_data_li:
        if (table in whole_data) == True:
            whole_data_copy.remove(table)
        
    output = '<'.join(whole_data_copy)

    mid = remove_tag(output)
    final = remove_space(mid)
    return final

In [90]:
page_list_copy = page_list.copy()
len(page_list_copy)

2873

In [93]:
text_lists = []
error_lists = []
for process, g in enumerate(page_list_copy):
    print("Process : {} | Total : {}".format(process+1,len(page_list_copy)),end='\r')
    try:
        text_lists.append(get_text_only(g))
    except ValueError:
        error_lists.append(process) 
        continue

In [94]:
len(text_lists)

2853

In [152]:
error_lists    # 20개

[85,
 254,
 255,
 356,
 358,
 360,
 366,
 367,
 368,
 527,
 675,
 1406,
 1416,
 1417,
 1418,
 1690,
 1809,
 1819,
 1831,
 1832]

In [96]:
for i in error_lists:
    text_lists.insert(i, np.nan)

In [97]:
len(text_lists)

2873

### error_lists에 있는 error가 어떤 회사의 경우인지 기록해놓고 수정해야 함!!

In [98]:
for i in error_lists:
    print(result_copy.crp_nm[i])

DGB금융지주
LG
LG
LS
LS
LS
LS산전
LS산전
LS산전
SK하이닉스
금호산업
삼성증권
삼성카드
삼성카드
삼성카드
알바이오
우리금융지주
우리은행
웅진에너지
웅진에너지


In [102]:
# DGB금융지주의 경우,
result_copy.iloc[85,:]['url']

'http://dart.fss.or.kr/report/viewer.do?rcpNo=20190401004795&dcmNo=6616826&eleId=18&offset=2808432&length=129886&dtd=dart3.xsd'

In [103]:
page_list_copy[85]

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html style="border:0">
<head>
<title></title>
<meta content="IE=EmulateIE8" http-equiv="X-UA-Compatible"/>
<meta content="gdi" http-equiv="X-UA-TextLayoutMetrics"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="/css/report_xml.css" rel="stylesheet" type="text/css"/>
</head>
<body bgcolor="#FFFFFF">
<p><br/></p>
<p class="section-1"><a name="toc1">IV. 이사의 경영진단 및 분석의견</a></p>
<p><br/></p>
<p><span style="font-size:16pt;line-height:1.6em;color:#003EFF;font-weight:bold;">1. 예측정보에 대한 주의사항<br/></span><br/>당사가 동 사업보고서에서 미래에 발생할 것으로 예상ㆍ예측한 활동, 사건 또는 현상은, 당해 공시서류 작성시점의 사건 및 재무성과에 대하여 회사의 견해를 반영한 것입니다. <br/>동 예측정보는 미래 사업환경과 관련된 다양한 가정에 기초하고 있으며, 동 가정들은결과적으로 부정확한 것으로 판명될 수도 있습니다. <br/>또한, 이러한 가정들에는 예측정보에서 기재한 예상치와 실제 결과 간에  중요한 차이를 초래할 수 있는 위험, 불확실성 및 기타 요인을 포함하고 있습니다. 이러한 중요한 차이를 초래할 수 있는 요인에는 회사 내부경영과 관련된 요인과 외부환경에 관한요인이 포함되어 있으며, 이에 한하지 않습니다.<br/>당사는 동 예측정

In [105]:
result_copy['str'] = text_lists

In [106]:
result_copy

,crp_nm,crp_cd,연도,rpt_nm,rcp_no,rcp_dt,rmk,url,str
0,AK홀딩스,6840,2018,사업보고서 (2018.12),20190401004433,20190401,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서...
1,AK홀딩스,6840,2017,사업보고서 (2017.12),20180330002417,20180330,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...
2,AK홀딩스,6840,2016,사업보고서 (2016.12),20170331003282,20170331,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...
3,AK홀딩스,6840,2015,사업보고서 (2015.12),20160330003602,20160330,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...
4,AK홀딩스,6840,2014,사업보고서 (2014.12),20150331003839,20150331,정연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...
...,...,...,...,...,...,...,...,...,...
2868,휴켐스,69260,2013,사업보고서 (2013.12),20140331003099,20140331,정연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 사업보고서에는 미래...
2869,휴켐스,69260,2012,사업보고서 (2012.12),20130401000720,20130401,정연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 사업보고서에는 미래...
2870,휴켐스,69260,2011,사업보고서 (2011.12),20120330000492,20120330,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 사업보고서에는 미래...
2871,휴켐스,69260,2010,[첨부추가]사업보고서 (2010.12),20110331002418,20110331,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견휴켐스㈜ 이사회는 2010년 1월 1일부터 2010...


In [107]:
result_copy.iloc[:,8]

0       IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서...
1       V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...
2       V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...
3       V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...
4       V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...
                              ...                        
2868    V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 사업보고서에는 미래...
2869    V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 사업보고서에는 미래...
2870    V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 사업보고서에는 미래...
2871    V. 이사의 경영진단 및 분석의견휴켐스㈜ 이사회는 2010년 1월 1일부터 2010...
2872    V. 이사의 경영진단 및 분석의견휴켐스㈜ 이사회는 2009년 1월 1일부터 2009...
Name: str, Length: 2873, dtype: object

In [118]:
# 의미없는 태그 삭제하는 함수

def extract_tags(tag, tokens, tokens_df):
    
    tokens_df_li = list(tokens_df[1])
    if (tag in tokens_df_li) == True:
        pass
    else:
        return # if tag doesn't exist in tokens, escape def
    
    indices = []
    for index in range(len(tokens)):
        if tokens[index][1] == tag:
            indices.append(index)

    indices.reverse()
    for index in indices:
        tokens.pop(index)

#     # 제대로 삭제됐는지 확인
#     cnt = 0
#     for i in range(len(tokens)):
#         if tokens[i][1] == tag:
#             cnt += 1
#     print(len(indices))
#     print(cnt)   # 0이어야 함
#     print(len(m))

In [119]:
def extract_tags_bundle():
    extract_tags('SN', m, me)       # 숫자 SN
    extract_tags('IC', m, me)       # 감탄사 IC 
    extract_tags('EF', m, me)       # 종결어미 EF (습니다)
    extract_tags('NNBC', m, me)       # 단위를 나타내는 명사 NNBC (년,월,원)
    extract_tags('NR', m, me)       # 수사 NR (억,천,백만)
    extract_tags('SC', m, me)       # 구분자 SC ( , : · )
    extract_tags('SF', m, me)       # 마침표, 물음표, 느낌표 SF ( . )
    extract_tags('SSC', m, me)       # 닫는 괄호 SSC ), ]
    extract_tags('SSO', m, me)       # 여는 괄호 SSO (, [
    extract_tags('SY', m, me)       # 기타 기호 SY(→, ", -, %, &, *), ; )
    extract_tags('JC', m, me)       # 접속 조사 JC (와, 과, 이나)

In [110]:
result_copy.iloc[85:90, :]

,crp_nm,crp_cd,연도,rpt_nm,rcp_no,rcp_dt,rmk,url,str
85,DGB금융지주,139130,2018,사업보고서 (2018.12),20190401004795,20190401,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,NaN
86,DGB금융지주,139130,2017,사업보고서 (2017.12),20180402004848,20180402,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...
87,DGB금융지주,139130,2016,사업보고서 (2016.12),20170331003962,20170331,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...
88,DGB금융지주,139130,2015,사업보고서 (2015.12),20160330001005,20160330,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...
89,DGB금융지주,139130,2014,사업보고서 (2014.12),20150331001266,20150331,정연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...


In [113]:
type(result_copy.iloc[85,8])

float

In [120]:
result_copy['str_tokens'] = np.nan
result_copy

,crp_nm,crp_cd,연도,rpt_nm,rcp_no,rcp_dt,rmk,url,str,str_tokens
0,AK홀딩스,6840,2018,사업보고서 (2018.12),20190401004433,20190401,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서...,NaN
1,AK홀딩스,6840,2017,사업보고서 (2017.12),20180330002417,20180330,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...,NaN
2,AK홀딩스,6840,2016,사업보고서 (2016.12),20170331003282,20170331,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...,NaN
3,AK홀딩스,6840,2015,사업보고서 (2015.12),20160330003602,20160330,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...,NaN
4,AK홀딩스,6840,2014,사업보고서 (2014.12),20150331003839,20150331,정연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...,NaN
...,...,...,...,...,...,...,...,...,...,...
2868,휴켐스,69260,2013,사업보고서 (2013.12),20140331003099,20140331,정연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 사업보고서에는 미래...,NaN
2869,휴켐스,69260,2012,사업보고서 (2012.12),20130401000720,20130401,정연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 사업보고서에는 미래...,NaN
2870,휴켐스,69260,2011,사업보고서 (2011.12),20120330000492,20120330,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 사업보고서에는 미래...,NaN
2871,휴켐스,69260,2010,[첨부추가]사업보고서 (2010.12),20110331002418,20110331,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견휴켐스㈜ 이사회는 2010년 1월 1일부터 2010...,NaN


In [121]:
for s in range(len(result_copy)):
    print("Process : {} | Total : {}".format(s+1,len(result_copy)),end='\r')
    str_ = result_copy.iloc[s, 8]
    if type(str_) == str:
        mecab = Mecab()
        token = mecab.pos(str_)
        mecab.sent_words(token)
        mecab.replace_synonyms(token)
        m = mecab.lemmatize(token)    # 리스트 형태로 출력
        me = pd.DataFrame(m)
        extract_tags_bundle()
        token_all = []
        for i in range(len(m)):
            if m[i][0] == 'lt' or m[i][0] == 'gt':      # m을 출력해보니 '<', '>'가 'lt', 'gt'로 변형되는 경우가 생겨 제거해줌
                pass
            else:
                token_all.append(m[i][0])
        sum_token = ' '.join(token_all)         # token_all 리스트에 담긴 토큰화한 단어들을 ' '(공백)으로 구분해서 sum_token에 저장
        result_copy.iloc[s, 9] = sum_token
    else:
        continue

In [122]:
result_copy

,crp_nm,crp_cd,연도,rpt_nm,rcp_no,rcp_dt,rmk,url,str,str_tokens
0,AK홀딩스,6840,2018,사업보고서 (2018.12),20190401004433,20190401,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서...,IV 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 당사 가 동 ...
1,AK홀딩스,6840,2017,사업보고서 (2017.12),20180330002417,20180330,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...,V 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 당사 가 동 사...
2,AK홀딩스,6840,2016,사업보고서 (2016.12),20170331003282,20170331,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...,V 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 당사 가 동 사...
3,AK홀딩스,6840,2015,사업보고서 (2015.12),20160330003602,20160330,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...,V 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 당사 가 동 사...
4,AK홀딩스,6840,2014,사업보고서 (2014.12),20150331003839,20150331,정연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...,V 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 당사 가 동 사...
...,...,...,...,...,...,...,...,...,...,...
2868,휴켐스,69260,2013,사업보고서 (2013.12),20140331003099,20140331,정연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 사업보고서에는 미래...,V 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 본 사업보고서 ...
2869,휴켐스,69260,2012,사업보고서 (2012.12),20130401000720,20130401,정연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 사업보고서에는 미래...,V 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 본 사업보고서 ...
2870,휴켐스,69260,2011,사업보고서 (2011.12),20120330000492,20120330,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 사업보고서에는 미래...,V 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 본 사업보고서 ...
2871,휴켐스,69260,2010,[첨부추가]사업보고서 (2010.12),20110331002418,20110331,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견휴켐스㈜ 이사회는 2010년 1월 1일부터 2010...,V 이사 의 경영진단 및 분석 의견 휴켐스 이사회 는 일 부터 일 까지 제 기 사업...


In [123]:
result_copy.to_csv(r'C:\Users\PC\OneDrive\Data Science\Weak-supervision\생존편향 제거\완료된 데이터\Lv.1_dongchan.csv', encoding='euc-kr')

In [124]:
# 09 ~ 18년도 이외의 보고서가 있는지 확인
result_copy['연도'].value_counts()

2013    303
2018    295
2017    294
2014    291
2015    290
2011    288
2016    288
2012    286
2010    280
2009    258
Name: 연도, dtype: int64

In [27]:
lv2_df

array([[       nan, 0.45548454,        nan, ...,        nan, 0.45548454,
        0.45548454],
       [       nan,        nan,        nan, ..., 0.45548454, 0.45548454,
               nan],
       [       nan, 0.45548454, 0.45548454, ..., 0.45548454, 0.45548454,
               nan],
       ...,
       [0.45548454, 0.45548454,        nan, ...,        nan, 0.45548454,
               nan],
       [0.45548454, 0.45548454, 0.45548454, ..., 0.45548454,        nan,
               nan],
       [       nan, 0.45548454, 0.45548454, ..., 0.45548454,        nan,
        0.45548454]])

In [ ]:
lv2_df.loc[univ_mat.columns]

### lv2_df는 20100101 ~ 20191231의 기간동안 KOSPI200에 한 번이라도 편입된 적 있는 기업들을 columns에 담아 

### 전년도&금년도 similarity scores를 담은 데이터프레임

In [127]:
lv1_df = pd.read_csv(r'C:\Users\PC\OneDrive\Data Science\Weak-supervision\생존편향 제거\완료된 데이터\Lv.1_dongchan.csv', encoding='euc-kr')
lv1_df.drop('Unnamed: 0', axis=1, inplace=True)
lv1_df

,crp_nm,crp_cd,연도,rpt_nm,rcp_no,rcp_dt,rmk,url,str,str_tokens
0,AK홀딩스,6840,2018,사업보고서 (2018.12),20190401004433,20190401,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서...,IV 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 당사 가 동 ...
1,AK홀딩스,6840,2017,사업보고서 (2017.12),20180330002417,20180330,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...,V 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 당사 가 동 사...
2,AK홀딩스,6840,2016,사업보고서 (2016.12),20170331003282,20170331,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...,V 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 당사 가 동 사...
3,AK홀딩스,6840,2015,사업보고서 (2015.12),20160330003602,20160330,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...,V 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 당사 가 동 사...
4,AK홀딩스,6840,2014,사업보고서 (2014.12),20150331003839,20150331,정연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...,V 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 당사 가 동 사...
...,...,...,...,...,...,...,...,...,...,...
2868,휴켐스,69260,2013,사업보고서 (2013.12),20140331003099,20140331,정연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 사업보고서에는 미래...,V 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 본 사업보고서 ...
2869,휴켐스,69260,2012,사업보고서 (2012.12),20130401000720,20130401,정연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 사업보고서에는 미래...,V 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 본 사업보고서 ...
2870,휴켐스,69260,2011,사업보고서 (2011.12),20120330000492,20120330,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 사업보고서에는 미래...,V 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 본 사업보고서 ...
2871,휴켐스,69260,2010,[첨부추가]사업보고서 (2010.12),20110331002418,20110331,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견휴켐스㈜ 이사회는 2010년 1월 1일부터 2010...,V 이사 의 경영진단 및 분석 의견 휴켐스 이사회 는 일 부터 일 까지 제 기 사업...


In [137]:
lv1_df['rcp_no'].values

array([20190401004433, 20180330002417, 20170331003282, ...,
       20120330000492, 20110331002418, 20100312000778], dtype=int64)

In [138]:
lv1_df['rcp_no'] = list(map(str, lv1_df.rcp_no))
lv1_df['rcp_dt'] = list(map(str, lv1_df.rcp_dt))
lv1_df['연도'] = list(map(str, lv1_df['연도']))

In [140]:
lv1_df['rcp_no'].values

array(['20190401004433', '20180330002417', '20170331003282', ...,
       '20120330000492', '20110331002418', '20100312000778'], dtype=object)

In [244]:
lv1_df.sort_values('연도', inplace=True)
lv1_df

,idx,crp_nm,crp_cd,연도,rpt_nm,rcp_no,rcp_dt,rmk,url,str,str_tokens
1984,2873,휴켐스,69260,2009,[첨부추가]사업보고서 (2009.12),20100312000778,20100312,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견휴켐스㈜ 이사회는 2009년 1월 1일부터 2009...,V 이사 의 경영진단 및 분석 의견 휴켐스 이사회 는 일 부터 일 까지 제 기 사업...
47,64,CJ제일제당,97950,2009,사업보고서 (2009.12),20100331001982,20100331,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견[예측정보에 대한 주의 사항] 본 자료는 회사의...,V 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 보 자료 는 회...
63,84,DB하이텍,990,2009,[첨부추가]사업보고서 (2009.12),20100330001204,20100330,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 경영진단의 범위 회사의 당해 사업년도 (제57...,V 이사 의 경영진단 및 분석 의견 경영진단 의 범위 회사 의 당해 사업년도 제 경...
1889,2737,현대엘리베이터,17800,2009,[첨부추가]사업보고서 (2009.12),20100331002497,20100331,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견가. 예측정보에 대한 주의사항 당사가 동 사업보고서...,V 이사 의 경영진단 및 분석 의 견가 예측 정보 에 대한 주 의 사항 당사 가 동...
1154,1597,신성이엔지,686068,2009,사업보고서 (2009.12),20100331001861,20100331,NaN,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서에...,V 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 당사 가 동 사...
...,...,...,...,...,...,...,...,...,...,...,...
1592,2311,한국쉘석유,2960,2018,사업보고서 (2018.12),20190401001204,20190401,NaN,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 당사의 과거 이익구...,IV 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 당사 의 과거...
421,515,SK케미칼,285130,2018,[기재정정]사업보고서 (2018.12),20190404002228,20190404,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항동 사업보고서의 예측...,IV 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 동 사업보고서...
660,848,대우건설,47040,2018,사업보고서 (2018.12),20190401003410,20190401,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 이사의 경...,IV 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 당사 가 동 ...
391,485,SK네트웍스,1740,2018,사업보고서 (2018.12),20190401003606,20190401,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 동 사업보고서...,IV 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 당사 가 동 ...


In [253]:
lv1_df.iloc[10:15,:]

,idx,crp_nm,crp_cd,연도,rpt_nm,rcp_no,rcp_dt,rmk,url,str,str_tokens
1275,1767,영진약품,3520,2009,사업보고서 (2009.12),20100331000680,20100331,NaN,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견영진약품공업주식회사 이사회는 2009년 1월 1일부...,V 이사 의 경영진단 및 분석 의견 영진약품공업 주식회사 이사회 는 일 부터 일 까...
910,1230,무림페이퍼,9200,2009,[첨부추가]사업보고서 (2009.12),20100331000301,20100331,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 당사가 동 이사의 경...,V 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 당사 가 동 이...
837,1127,롯데쇼핑,23530,2009,[첨부추가]사업보고서 (2009.12),20100331001525,20100331,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 당사가 동 사업보고서...,V 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 당사 가 동 사...
1110,1510,세방전지,4490,2009,사업보고서 (2009.12),20100331001532,20100331,NaN,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 당사가 동 이사의 경...,V 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 당사 가 동 이...
1212,1666,아모레퍼시픽,90430,2009,[첨부추가]사업보고서 (2009.12),20100331001527,20100331,연,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 본 사업보고서 ...,V 이사 의 경영진단 및 분석 의견 예측 정보 에 대한 주 의 사항 당사 가 보 사...


In [141]:
lv2_df = pd.DataFrame(np.full(shape = (len(lv2_index), len(lv2_companies)), fill_value=np.nan), index = lv2_index, columns = lv2_companies)
lv2_df

NameError: name 'lv2_companies' is not defined

In [311]:
%%time

for process,corp_name in enumerate(lv2_companies) : 
    print("Process : {} | Total : {}".format(process+1,len(lv2_companies)),end='\r')
    # slicing the dataset by corp_name
    tmp_df = lv1_df[lv1_df.crp_nm == corp_name]
    
    # set-up the time range 
    min_yr = tmp_df['연도'].min()
    max_yr = tmp_df['연도'].max()
    
    # container for saving the value 
    # those lists should be refreshed every loop.
    sim_ls = []; year_ls = []
    
    # next_df : 금년도, pre_df : 전년도 ---> 전년도와 비교했을 때 금년도 사업보고서가 얼마나 유사한지
    for yr in range(min_yr,max_yr) :          
        pre_df = tmp_df[tmp_df['연도'] == yr]
        next_df = tmp_df[tmp_df['연도'] == yr+1]
        publish_date = tmp_df[tmp_df['연도'] == yr+1]['rcp_dt']
        if len(pre_df) == 0 or len(next_df) == 0:
            continue
#             we don't have data
#             for example, we have 2014 but don't 2015.
#             therfore we can't measure similarity between 2014,2015.
#             so the value in lv2 at 2015 should be nan
            
        else : 
            pre_token = pre_df.str_tokens.map(lambda x : x.split(" ")).values
            next_token = next_df.str_tokens.map(lambda x : x.split(" ")).values   
            if type(pre_token[0]) == list and type(next_token[0]) == list :              ## [0]은 왜 해주는건지?
                sim = len(set(next_token[0])&set(pre_token[0]))/len(set(next_token[0]))
            else : 
                 sim = len(set(next_token)&set(pre_token))/len(set(next_token))
        year_ls.append(publish_date.values[0])
        sim_ls.append(sim)
        
    assert len(year_ls) == len(sim_ls)
    lv2_df.loc[year_ls,corp_name] = sim_ls    

KeyboardInterrupt: 

In [313]:
%%time
e_list = []
e_list1 = []
for process,corp_name in enumerate(lv2_companies) : 
    print("Process : {} | Total : {}".format(process+1,len(lv2_companies)),end='\r')
    try:
        # slicing the dataset by corp_name
        tmp_df = lv1_df[lv1_df.crp_nm == corp_name]
    
        # set-up the time range 
        min_yr = tmp_df['연도'].min()
        max_yr = tmp_df['연도'].max()
    
        # container for saving the value 
        # those lists should be refreshed every loop.
        sim_ls = []; year_ls = []
        # next_df : 금년도, pre_df : 전년도 ---> 전년도와 비교했을 때 금년도 사업보고서가 얼마나 유사한지
        for yr in range(min_yr,max_yr) :          
            pre_df = tmp_df[tmp_df['연도'] == yr]
            next_df = tmp_df[tmp_df['연도'] == yr+1]
            publish_date = tmp_df[tmp_df['연도'] == yr+1]['rcp_dt']
            if len(pre_df) == 0 or len(next_df) == 0:
                continue
    
#             we don't have data
#             for example, we have 2014 but don't 2015.
#             therfore we can't measure similarity between 2014,2015.
#             so the value in lv2 at 2015 should be nan
            
            else : 
                pre_token = pre_df.str_tokens.map(lambda x : x.split(" ")).values
                next_token = next_df.str_tokens.map(lambda x : x.split(" ")).values   
                if type(pre_token[0]) == list and type(next_token[0]) == list :              ## [0]은 왜 해주는건지?
                    sim = len(set(next_token[0])&set(pre_token[0]))/len(set(next_token[0]))
                else : 
                     sim = len(set(next_token)&set(pre_token))/len(set(next_token))
            year_ls.append(publish_date.values[0])
            sim_ls.append(sim)
        assert len(year_ls) == len(sim_ls)
        lv2_df.loc[year_ls,corp_name] = sim_ls   
    except TypeError:
        e_list.append(corp_name) 
        continue
    except AttributeError:
        e_list1.append(corp_name)
        continue
     

KeyboardInterrupt: 

In [293]:
e_list

['DGB금융지주', '동일산업', '신세계인터내셔날', '알루코', '이수화학']

In [294]:
e_list1

['LG', 'LS', 'LS산전', 'SK하이닉스', '삼성증권', '삼성카드', '우리금융지주', '우리금융지주']

In [263]:
tmp_df = lv1_df[lv1_df.crp_nm == 'GKL']
 # set-up the time range 
min_yr = tmp_df['연도'].min()
max_yr = tmp_df['연도'].max()
    
    # container for saving the value 
    # those lists should be refreshed every loop.

    
    # next_df : 금년도, pre_df : 전년도 ---> 전년도와 비교했을 때 금년도 사업보고서가 얼마나 유사한지
for yr in range(min_yr,max_yr) :          
    pre_df = tmp_df[tmp_df['연도'] == yr]
    next_df = tmp_df[tmp_df['연도'] == yr+1]
    publish_date = tmp_df[tmp_df['연도'] == yr+1]['rcp_dt']
    print(publish_date)
    if len(pre_df) == 0 or len(next_df) == 0:
        continue

67    20160328
Name: rcp_dt, dtype: int64
66    20170330
Name: rcp_dt, dtype: int64
65    20180402
Name: rcp_dt, dtype: int64
64    20190705
Name: rcp_dt, dtype: int64


In [ ]:
lv2_df.mean()